In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet, Lasso
from statsmodels.tsa.ar_model import AutoReg
from sklearn.cross_decomposition import PLSRegression

from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse

from scipy.stats import norm
from statsmodels.tsa.stattools import acovf

import warnings
from tqdm import tqdm
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

In [2]:
data = pd.read_csv('data/df_processed.csv')
data = data.rename(columns = {'inflaton_rate' : 'inflation_rate'})
data = data.drop(columns = ['interest_rate'])                      # Remove interest rate 
#data = data.drop(columns = ['MOEX'])                      # Remove MOEX 
data

,month-year,inflation_rate,CPI,unemplyment_rate,GDP,fx_eur_rub,fx_usd_rub,oil_price,gas_price,gold_price,exports,imports,external_debt,chicken_eggs_10pcs,granulated_sugar_1kg,wheat_flour_1kg,avg_monthly_salary,sanctions,topic_0,topic_1,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,topic_2,topic_20,topic_21,topic_22,topic_23,topic_24,topic_25,topic_26,topic_27,topic_28,topic_29,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,inflaton_rate_lag_1,interest_rate_lag_1,CPI_lag_1,unemplyment_rate_lag_1,GDP_lag_1,fx_eur_rub_lag_1,fx_usd_rub_lag_1,oil_price_lag_1,gas_price_lag_1,gold_price_lag_1,exports_lag_1,imports_lag_1,external_debt_lag_1,chicken_eggs_10pcs_lag_1,granulated_sugar_1kg_lag_1,wheat_flour_1kg_lag_1,avg_monthly_salary_lag_1,sanctions_lag_1,topic_0_lag_1,topic_1_lag_1,topic_10_lag_1,topic_11_lag_1,topic_12_lag_1,topic_13_lag_1,topic_14_lag_1,topic_15_lag_1,topic_16_lag_1,topic_17_lag_1,topic_18_lag_1,topic_19_lag_1,topic_2_lag_1,topic_20_lag_1,topic_21_lag_1,topic_22_lag_1,topic_23_lag_1,topic_24_lag_1,topic_25_lag_1,topic_26_lag_1,topic_27_lag_1,topic_28_lag_1,topic_29_lag_1,topic_3_lag_1,topic_4_lag_1,topic_5_lag_1,topic_6_lag_1,topic_7_lag_1,topic_8_lag_1,topic_9_lag_1
0,2011-06,-0.17,0.2,-0.80,1410.022767,0.294486,0.072509,-5.068030,0.179903,16.981168,-183.4091,-612.7362,29759.00,-4.40,-0.99,-0.23,1358.0,0.0,0.010965,0.020755,0.042945,-0.024100,-0.000870,0.003960,-0.005177,-0.008439,0.000393,0.003873,-0.010572,0.003308,0.001046,-0.025016,-0.036764,0.007963,-0.000652,0.004178,0.020450,-0.004525,0.013010,0.013621,-0.009179,-0.011608,0.003003,0.045036,0.000611,0.004090,0.005048,0.007310,-0.02,3.0,0.6,0.20,0.000000,-0.492550,-0.211921,-8.681834,0.064493,29.149286,-1509.6119,999.5400,0.00,-6.87,-1.62,-0.19,260.0,0.0,-0.015291,-0.008705,0.027149,0.046140,-0.005383,-0.002118,-0.005183,-0.041349,0.019905,0.002262,-0.028721,0.017041,0.008994,0.000089,0.009740,0.002262,0.004725,-0.010566,0.040724,0.000401,-0.009106,-0.012082,0.008994,0.020315,-0.003121,-0.004693,-0.013285,0.006787,0.005127,-0.015692
1,2011-07,-0.41,0.0,-0.30,0.000000,-0.367301,-0.082742,1.051863,-0.162795,49.129550,-2030.2035,-298.6864,-29759.00,0.34,0.65,-0.07,-539.0,0.0,-0.003413,-0.010686,0.042254,-0.009634,0.017642,0.015669,-0.001440,0.038308,0.001051,-0.003788,0.032633,-0.011593,-0.011593,-0.003716,0.018016,-0.007878,0.003255,0.008324,0.014085,0.007042,-0.017195,-0.025836,0.003327,-0.008828,-0.004925,-0.014488,-0.001599,0.007042,-0.011665,0.007114,-0.17,0.0,0.2,-0.80,1410.022767,0.294486,0.072509,-5.068030,0.179903,16.981168,-183.4091,-612.7362,29759.00,-4.40,-0.99,-0.23,1358.0,0.0,0.010965,0.020755,0.042945,-0.024100,-0.000870,0.003960,-0.005177,-0.008439,0.000393,0.003873,-0.010572,0.003308,0.001046,-0.025016,-0.036764,0.007963,-0.000652,0.004178,0.020450,-0.004525,0.013010,0.013621,-0.009179,-0.011608,0.003003,0.045036,0.000611,0.004090,0.005048,0.007310
2,2011-08,-0.85,-0.3,0.40,0.000000,1.164849,0.860332,-10.999630,-0.369685,181.366300,2634.6997,2707.3087,0.00,2.97,-0.64,-0.12,-547.0,0.0,0.004574,-0.016487,0.025701,0.111831,-0.002457,-0.011880,0.004651,-0.049416,0.002150,0.004662,-0.009565,0.004574,0.006911,0.034959,-0.018900,-0.000011,-0.007053,0.004519,0.023364,-0.004706,0.006965,-0.014194,-0.018812,-0.052268,-0.007053,0.013580,-0.002501,0.021028,0.011660,-0.002446,-0.41,0.0,0.0,-0.30,0.000000,-0.367301,-0.082742,1.051863,-0.162795,49.129550,-2030.2035,-298.6864,-29759.00,0.34,0.65,-0.07,-539.0,0.0,-0.003413,-0.010686,0.042254,-0.009634,0.017642,0.015669,-0.001440,0.038308,0.001051,-0.003788,0.032633,-0.011593,-0.011593,-0.003716,0.018016,-0.007878,0.003255,0.008324,0.014085,0.007042,-0.017195,-0.025836,0.003327,-0.008828,-0.004925,-0.014488,-0.001599,0.007042,-0.011665,0.007114
3,2011-09,-0.95,0.0,-0.40,1310.761261,1.195111,1.861869,-0.730871,-0.134518,3.694409,-781.6325,-2594.2622,-12542.00,3.12,-3.83,-0.41,417.0,0.0,-0.009828,-0.002612,0.058957,-0.073331,-0.009759,-0.000895,-0.000275,0.015115,0

## HyperParams grid

In [3]:
random_forest_param_grid = {
#     'n_estimators' : [10, 30, 50, 100],
    'max_depth' : [None, 1, 10, 20],
    'random_state' : [42]
}

lasso_param_grid = {
    'alpha': [0.01, 0.1, 1, 10, 100]
    }

pls_param_grid = {
    'n_components': [5, 10, 20, 30, 40, 50, 60]
    }

elasticnet_param_grid = {
    'alpha': [0.01, 0.1, 1, 10, 100],
    'l1_ratio': [0.2, 0.5, 0.8]
}

horizon = 12
split_point = int(len(data) * 0.85)
print(split_point)

128


In [4]:
def test_a_model(data, split_point, horizon, model, param_grid):
    frame = data.copy()

    features = list(frame.drop(columns = ['month-year', 'inflation_rate']).columns)

    # Train and Test sets snapshot
    train_part = frame.iloc[ : split_point]
    test_part = frame.iloc[split_point : ]

    X_train = frame.iloc[ : split_point][features]
    y_train = frame.iloc[ : split_point]['inflation_rate'].values
    y_test = frame.iloc[split_point : split_point + horizon + 1]['inflation_rate'].to_list()

    cv = GridSearchCV(model, param_grid, scoring='neg_mean_squared_error', cv=5)
    cv.fit(X_train, y_train)
    best_model = cv.best_estimator_
    
    predictions = []

    # horizon (12 periods) predictions
    for h in range(horizon):
        test_step = frame.iloc[split_point+h:split_point+h+1][features] # takes 1 observation from test to predict
        
        if h > 0:
            test_step['inflaton_rate_lag_1'] = prediction # replaces inflaton_rate_lag_1 with last_prediction  
        
        prediction = best_model.predict(test_step)[0]     # makes 1 prediction
        predictions.append(prediction)                    # saves prediction
        
    return {
        'Best Model': best_model,
        'True Values': y_test,
        'Predictions': predictions,
        'Train Size' : len(X_train)
        }


In [5]:
def recursive_loop(data):
    all_results = {
        'RF':[],
        'LASSO':[],
        'ENET': [],
        'PLS' : [],
        'AR4':[]
        }

    for i in tqdm(range(split_point, len(data)-horizon)):

        # RandomForest
        rf_results = test_a_model(
            data        = data,
            split_point = i,
            horizon     = horizon,
            model       = RandomForestRegressor(),
            param_grid  = random_forest_param_grid,
        )
        all_results['RF'].append((i, rf_results))

        # LASSO
        lasso_results = test_a_model(
            data        = data,
            split_point = split_point,
            horizon     = horizon,
            model       = Lasso(),
            param_grid  = lasso_param_grid,
            )
        all_results['LASSO'].append((i, lasso_results))

        # Elastic-NET
        elastic_net_results = test_a_model(
            data        = data,
            split_point = split_point,
            horizon     = horizon,
            model       = ElasticNet(),
            param_grid  = elasticnet_param_grid,
            )
        all_results['ENET'].append((i, elastic_net_results))
        
        # PLSRegression
        pls_results = test_a_model(
            data        = data,
            split_point = split_point,
            horizon     = horizon,
            model       = PLSRegression(),
            param_grid  = pls_param_grid,
            )
        all_results['PLS'].append((i, pls_results))
        
        
        # AR4
        y_train = data.iloc[ : i]['inflation_rate'].values
        ar4_model = AutoReg(y_train, lags=4).fit()
        predictions = ar4_model.predict(start=len(y_train), end=len(y_train)+horizon)
        all_results['AR4'].append((i, {'Predictions': predictions, 'Train Size': len(y_train)}))
    return all_results

In [6]:
def newey_west_adjusted_dm_stat(loss_diff, horizon):
    """
    Computes the Newey-West adjusted Diebold-Mariano statistic.
    """
    T = len(loss_diff)
    mean_loss_diff = np.mean(loss_diff)
    autocovariances = acovf(loss_diff, fft=True)[:horizon]
    nw_variance = autocovariances[0] + 2 * np.sum((1 - np.arange(1, horizon) / horizon) * autocovariances[1:horizon])
    dm_stat = mean_loss_diff / np.sqrt(nw_variance / T)
    return dm_stat

def get_stats(all_results):
    train_sizes = []
    models = []
    horizons = []
    rmses = []
    rRMSEs = []
    dm_results = []
    significances = []
    for k in range(len(all_results['RF'])-1):
        for model in ['RF', 'LASSO', 'ENET', 'PLS','AR4']:
            for h in [3, 6, 9, 12]:

                if model == 'AR4':
                    rmse = np.sqrt(mse(all_results['RF'][k][1]['True Values'][:h], all_results[model][k][1]['Predictions'][:h]))
                    rRMSE = rmse / np.sqrt(mse(all_results['RF'][k][1]['True Values'][:h], all_results[model][k][1]['Predictions'][:h]))
                    train_sizes.append(all_results[model][k][0])
                    models.append(model)
                    horizons.append(h)
                    rmses.append(rmse)
                    rRMSEs.append(rRMSE)
                    dm_results.append(None)
                    significances.append(None)

                else:
                    rmse = np.sqrt(mse(all_results[model][k][1]['True Values'][:h], all_results[model][k][1]['Predictions'][:h]))
                    rRMSE = rmse / np.sqrt(mse(all_results[model][k][1]['True Values'][:h], all_results['AR4'][k][1]['Predictions'][:h]))

                    error_diff = np.array(all_results[model][k][1]['True Values'][:h]) - np.array(all_results[model][k][1]['Predictions'][:h])
                    error_ar4 = np.array(all_results[model][k][1]['True Values'][:h]) - np.array(all_results['AR4'][k][1]['Predictions'][:h])
                    loss_diff = error_diff**2 - error_ar4**2
                    dm_stat = newey_west_adjusted_dm_stat(loss_diff, h) 

                    if dm_stat > 0:
                        pval = 1 - norm.cdf(dm_stat)  # One-sided test
                    else:
                        pval = norm.cdf(dm_stat)

                    dm_result = np.round(dm_stat, 5) 
                    significance = "*" if pval < 0.05 else "" 
                    significance = "**" if pval < 0.01 else significance 
                    significance = "***" if pval < 0.001 else significance

                    train_sizes.append(all_results[model][k][0])
                    models.append(model)
                    horizons.append(h)
                    rmses.append(rmse)
                    rRMSEs.append(rRMSE)
                    dm_results.append(dm_result)
                    significances.append(significance)


    results = pd.DataFrame(data={
        'model': models,
        'Train Size': train_sizes,
        'horizon (months)': horizons,
        'RMSE': rmses,
        'rRMSE': rRMSEs,
        'DM-test' : dm_results,
        'Significance' : significances})

    return results   

# Run models on different set of features

In [7]:
me_results = recursive_loop(data[[ _ for _ in data.columns if 'topic_' not in _]])

topics_results = recursive_loop(data[['month-year', 'inflation_rate', 'inflaton_rate_lag_1']+[ _ for _ in data.columns if 'topic_' in _]])

me_and_topics_results = recursive_loop(data)

100%|███████████████████████████████████████████████████████████████████████████| 11/11 [01:19<00:00,  7.23s/it]


In [8]:
me_stats = get_stats(me_results)
topics_stats = get_stats(topics_results)
me_and_topics = get_stats(me_and_topics_results)

In [9]:
me_stats[me_stats['Train Size']==130]

,model,Train Size,horizon (months),RMSE,rRMSE,DM-test,Significance
40,RF,130,3,1.035615,0.202128,-5.35015,***
41,RF,130,6,0.805184,0.208714,-2.52600,**
42,RF,130,9,0.693036,0.211422,-2.19924,*
43,RF,130,12,2.235146,0.611401,-2.20334,*
44,LASSO,130,3,1.016711,0.242380,-1.90759,*
45,LASSO,130,6,0.762822,0.231059,-2.15252,*
46,LASSO,130,9,0.691837,0.241263,-2.22727,*
47,LASSO,130,12,0.603997,0.240247,-2.01212,*
48,ENET,130,3,1.319028,0.314451,-1.81775,*
49,ENET,130,6,0.962264,0.291470,-2.13491,*


In [10]:
topics_stats[topics_stats['Train Size']==130]

,model,Train Size,horizon (months),RMSE,rRMSE,DM-test,Significance
40,RF,130,3,1.441951,0.281436,-4.46961,***
41,RF,130,6,1.260497,0.326737,-2.29842,*
42,RF,130,9,1.186392,0.361927,-1.98116,*
43,RF,130,12,2.552348,0.698169,-1.66555,*
44,LASSO,130,3,4.298634,1.024778,0.07116,
45,LASSO,130,6,3.119556,0.944915,-0.27258,
46,LASSO,130,9,2.594473,0.904764,-0.63668,
47,LASSO,130,12,2.255699,0.897232,-0.81001,
48,ENET,130,3,4.297555,1.024520,0.07042,
49,ENET,130,6,3.119026,0.944754,-0.27340,


In [11]:
me_and_topics[me_and_topics['Train Size']==130]

,model,Train Size,horizon (months),RMSE,rRMSE,DM-test,Significance
40,RF,130,3,0.984294,0.192112,-5.17184,***
41,RF,130,6,0.789641,0.204685,-2.49630,**
42,RF,130,9,0.684189,0.208723,-2.18261,*
43,RF,130,12,2.258246,0.617720,-2.13147,*
44,LASSO,130,3,1.016711,0.242380,-1.90759,*
45,LASSO,130,6,0.762822,0.231059,-2.15252,*
46,LASSO,130,9,0.691837,0.241263,-2.22727,*
47,LASSO,130,12,0.603997,0.240247,-2.01212,*
48,ENET,130,3,1.319028,0.314451,-1.81775,*
49,ENET,130,6,0.962264,0.291470,-2.13491,*


# Joint report

In [12]:
def make_report(train_size):
    m = me_stats[me_stats['Train Size']==train_size][['model', 'horizon (months)', 'rRMSE']]
    n = topics_stats[topics_stats['Train Size']==train_size][['model', 'horizon (months)', 'rRMSE']]
    m_n = me_and_topics[me_and_topics['Train Size']==train_size][['model', 'horizon (months)', 'rRMSE']]
    
    m = m.rename(columns = {'rRMSE' : 'rRMSE (M)'})
    n = n.rename(columns = {'rRMSE' : 'rRMSE (N)'})
    m_n = m_n.rename(columns = {'rRMSE' : 'rRMSE (M+N)'})
    
    report = m.merge(n, how = 'left', on = ['model', 'horizon (months)'])
    report = report.merge(m_n, how = 'left', on = ['model', 'horizon (months)'])
    report = report[report['model'] != 'AR4']
    
    report['% Improvement'] = np.round(100 * (report['rRMSE (M+N)'] - report['rRMSE (M)']) / report['rRMSE (M)'], 2)
    return report


In [13]:
report = make_report(130)
report

,model,horizon (months),rRMSE (M),rRMSE (N),rRMSE (M+N),% Improvement
0,RF,3,0.202128,0.281436,0.192112,-4.96
1,RF,6,0.208714,0.326737,0.204685,-1.93
2,RF,9,0.211422,0.361927,0.208723,-1.28
3,RF,12,0.611401,0.698169,0.617720,1.03
4,LASSO,3,0.242380,1.024778,0.242380,0.00
5,LASSO,6,0.231059,0.944915,0.231059,0.00
6,LASSO,9,0.241263,0.904764,0.241263,0.00
7,LASSO,12,0.240247,0.897232,0.240247,0.00
8,ENET,3,0.314451,1.024520,0.314451,0.00
9,ENET,6,0.291470,0.944754,0.291470,0.00
